# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding='utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding. 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Image of the denormalized data in <font color=red>**event_datafile_new.csv**</font><br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)



#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)
    


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Query 1 Description: In this query, I used session_id as the partition key and item_in_session as my clustering key. Each partition is uniquely identified by session_id while item_in_session was used to uniquely identify the rows within a partition to sort the data by the value of num.

#### Artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
query1 = 'SELECT artist,song,length FROM session_history WHERE session_id = 338 AND item_in_session = 4'

try:
    session.execute("""CREATE TABLE IF NOT EXISTS session_history (session_id int, item_in_session int, artist text, song text, length float,
    PRIMARY KEY ((session_id), item_in_session))""")
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO session_history (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
rows = session.execute(query1)
for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2 Description: In this query, I used user_id and session_id as the partition keys and item_in_session as my clustering key. Each partition is uniquely identified by user_id, session_id while item_in_session was used to uniquely identify the rows within a partition to sort the data by the value of num.

#### Artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
query2 = 'SELECT artist,song,user FROM song_history WHERE user_id = 10 AND session_id = 182'

try:
    session.execute("""CREATE TABLE IF NOT EXISTS song_history (user_id int, session_id int, item_in_session int, artist text, song text, user text,
    PRIMARY KEY ((user_id, session_id), item_in_Session))""")
except Exception as e:
    print(e)

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_history (user_id, session_id, item_in_session, artist, song, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1]+' '+line[4]))

In [13]:
rows = session.execute(query2)
for row in rows:
    print(row.artist,row.song,row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3 Description: In this query, I used song and user_id as the partition keys to uniquely identify each user, song is then used to filter the values using the WHERE clause.

#### User (first and last) in music app history who listened to the song 'All Hands Against His Own'

In [14]:
query3 = "SELECT user FROM user_history WHERE song = 'All Hands Against His Own'"

try:
    session.execute("""CREATE TABLE IF NOT EXISTS user_history (song text, user_id int, user text,
    PRIMARY KEY (song, user_id))""")
except Exception as e:
    print(e)

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_history (song, user_id, user)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1]+' '+line[4]))

TypeError: not all arguments converted during string formatting

In [ ]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user)

### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute("DROP TABLE session_history")
    session.execute("DROP TABLE song_history")
    session.execute("DROP TABLE user_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()